## Семинар 1 Индекс

## Intro

##  Индекс 

Сам по себе индекс - это просто формат хранения данных, он не может осуществлять поиск. Для этого необходимо добавить к нему определенную метрику. Это может быть что-то простое типа булева поиска, а может быть что-то более специфическое или кастомное под задачу.

Давайте посмотрим, что полезного можно вытащить из самого индекса.    
По сути, индекс - это информация о частоте встречаемости слова в каждом документе.   
Из этого можно понять, например:
1. какое слово является самым часто употребимым / редким
2. какие слова встречаются всегда вместе - так можно парсить твиттер, fb, форумы и отлавливать новые устойчивые выражения в речи
3. как эти документы кластеризуются по N тематикам согласно словам, которые в них упоминаются 

## __Задача__: 

**Data:** Коллекция субтитров сезонов Друзьей. Одна серия - один документ.

**To do:** Постройте небольшой модуль поискового движка, который сможет осуществлять поиск по коллекции документов.
На входе запрос и проиндексированная коллекция (в том виде, как посчитаете нужным), на выходе отсортированный по релевантности с запросом список документов коллекции. 

Релизуйте:
    - функцию препроцессинга данных
    - функцию индексирования данных
    - функцию метрики релевантности 
    - собственно, функцию поиска

[download_friends_corpus](https://yadi.sk/d/yVO1QV98CDibpw)

Напоминание про defaultdict: 
> В качестве multiple values словаря рекомендую использовать ``` collections.defaultdict ```                          
> Так можно избежать конструкции ``` dict.setdefault(key, default=None) ```

In [0]:
### _check : в коллекции должно быть около 165 файлов

In [1]:
pip install pymorphy2

     |████████████████████████████████| 51kB 2.0MB/s 
     |████████████████████████████████| 7.1MB 8.0MB/s 


С помощью обратного индекса посчитайте:  


a) какое слово является самым частотным

b) какое самым редким

c) какой набор слов есть во всех документах коллекции

d) какой сезон был самым популярным у Чендлера? у Моники?

e) кто из главных героев статистически самый популярный? 


In [0]:
import os
import re
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
os.chdir('gdrive/My Drive/Colab Notebooks/friends')

In [5]:
for file in os.listdir(os.getcwd()):
  print(file)

Friends - season 3
Friends - season 7
Friends - season 1
Friends - season 5
Friends - season 2
Friends - season 6
Friends - season 4


In [6]:
top = os.getcwd()
top

'/content/gdrive/My Drive/Colab Notebooks/friends'

In [0]:
folder = []
for i in os.walk(top):
  folder.append(i)

In [0]:
allTexts = []
allNames = []
for address, dirs, files in folder:
  for file in files:
    allNames.append(file)
    with open(address+'/'+file, 'r') as f:
      text = f.read()
      allTexts.append(text)

In [28]:
len(allTexts)

165

In [0]:
allWordsAllTexts = []
for text in allTexts:
  allWordsInOneText = []
  if text != "":
    for word in text.split():
      word = re.sub('[.,-;:?!@#$%^&()_+=—\ufeff–"…«»>wwwtvsubtitlesnet]', '', word).lower()
      if word != "":
        p = morph.parse(word)[0]
        if p not in allWordsInOneText:
          allWordsInOneText.append(p.normal_form)
    allWordsAllTexts.append(allWordsInOneText)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [0]:
vectorizer = CountVectorizer()

In [0]:
arrayString = []
for text in allWordsAllTexts:
  s = ' '.join(text)
  arrayString.append(s)

In [0]:
X = vectorizer.fit_transform(arrayString)

In [0]:
Matrix = X.toarray()

In [16]:
print(Matrix)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [0]:
import pandas as pd
df = pd.DataFrame(Matrix, index=allNames, columns=vectorizer.get_feature_names())

In [18]:
df.head()

,aa,ad,afr,aga,ahh,ar,ay,aнгел,bay,bhd,bodgo,bojor,car,cat,cd,cdпроигрыватель,ch,cha,chck,ck,co,cy,cамый,cочельник,cтарый,cтоп,da,dac,dayar,dck,dcky,dcory,dd,dh,dings,do,doo,dry,dьzm,echa,...,ямагучить,ямочка,ямс,ямснямнямс,ян,январь,янки,янкиз,янковский,японец,япония,японский,япросить,яркий,ярко,ярковата,ярлык,ярлычок,ярмарка,ярость,ясердцеросс,ясмина,ясно,ясность,ясный,ятебеееотдать,ято,яхта,ящерица,ящик,ящичек,ёй,ёкнуть,ёлка,ёлочный,ёпэрэсотэ,ёрл,ёрш,ёршик,ёще
Friends - 3x05 - The One With Frank Jr..ru.txt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
Friends - 3x03 - The One With The Jam.ru.txt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Friends - 3x02 - The One Where No One's Ready.ru.txt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Friends - 3x01 - The One With The Princess Leia Fantasy.ru.txt,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Friends - 3x04 - The One With The Metaphorical Tunnel.ru.txt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Поисковик**

In [0]:
from collections import Counter
import operator
import math

In [0]:
def tokenize(doc):
  allWordsInOneText = []
  text = doc.split()
  for word in text:
    word = re.sub('[.,-;:?!@#$%^&()_+=—\ufeff–"…«»>wwwtvsubtitlesnet]', '', word).lower()
    if word != "":
      p = morph.parse(word)[0]
      allWordsInOneText.append(p.normal_form)
  return allWordsInOneText

In [0]:
def build_terms(corpus):
    terms = {}
    current_index = 0
    for doc in corpus:
        for word in tokenize(doc):
            if word not in terms:
                terms[word] = current_index
                current_index += 1
    return terms

In [0]:
def tf(document, terms):
    words = tokenize(document)
    total_words = len(words)
    doc_counter = Counter(words)
    for word in doc_counter:
        doc_counter[word] /= total_words
    tfs = [0 for _ in range(len(terms))]
    for term, index in terms.items():
        tfs[index] = doc_counter[term]
    return tfs

In [0]:
def _count_docs_with_word(word, docs):
    counter = 1
    for doc in docs:
        if word in doc:
            counter += 1
    return counter

In [0]:
# documents - это корпус
def idf(documents, terms):
    idfs = [0 for _ in range(len(terms))]
    total_docs = len(documents)
    for word, index in terms.items():
        docs_with_word = _count_docs_with_word(word, documents)
        idf = 1 + math.log10(total_docs / docs_with_word)
        idfs[index] = idf
    return idfs

In [0]:
def _merge_td_idf(tf, idf, terms):
    return [tf[i] * idf[i] for i in range(len(terms))]


def build_tfidf(corpus, document, terms):
    doc_tf = tf(document, terms)
    doc_idf = idf(corpus, terms)
    return _merge_td_idf(doc_tf, doc_idf, terms)


def cosine_similarity(vec1, vec2):
    def dot_product2(v1, v2):
        return sum(map(operator.mul, v1, v2))

    def vector_cos5(v1, v2):
        prod = dot_product2(v1, v2)
        len1 = math.sqrt(dot_product2(v1, v1))
        len2 = math.sqrt(dot_product2(v2, v2))
        return prod / (len1 * len2)

    return vector_cos5(vec1, vec2)

In [0]:
tf_idf_total = []
corpus = allTexts
terms = build_terms(corpus)

In [30]:
len(corpus)

165

In [0]:
for document in corpus:
  tf_idf_total.append(build_tfidf(corpus, document, terms))

In [0]:
query = "Он подарил мне цветы"

In [0]:
results = {}
query_tfidf = build_tfidf(corpus, query, terms)
for index, document in enumerate(tf_idf_total):
  results[index] = cosine_similarity(query_tfidf, document)

In [44]:
ld = list(results.items())
ld.sort(key=lambda i: i[1], reverse= True)
ld

[(13, 0.2752542859743394),
 (148, 0.24471401179804136),
 (133, 0.23290323882225525),
 (7, 0.23290087954364497),
 (94, 0.22910673441624954),
 (137, 0.22879058779910527),
 (1, 0.22125968103179117),
 (10, 0.21973359393728478),
 (23, 0.21839336357825587),
 (46, 0.21734952102205848),
 (54, 0.2169524068135041),
 (22, 0.21692765155774393),
 (37, 0.21689218055271234),
 (113, 0.2166221741453063),
 (81, 0.21441365055359124),
 (68, 0.2138874585331985),
 (151, 0.2138853573391441),
 (110, 0.2130222236381641),
 (8, 0.21277829270285625),
 (108, 0.21196390891406372),
 (149, 0.21159810933109938),
 (163, 0.21144027786533223),
 (164, 0.21126199475596663),
 (42, 0.21015920308428204),
 (150, 0.20964690328532032),
 (63, 0.20959084004177658),
 (153, 0.20785210768323883),
 (135, 0.20774321164955098),
 (30, 0.20672401103432556),
 (97, 0.2067183881231493),
 (160, 0.20593766518551734),
 (144, 0.20283250185744026),
 (93, 0.20266241032202806),
 (100, 0.20254230232391018),
 (11, 0.20223281636772597),
 (16, 0.202170

In [45]:
print('По вашему запросу лучший результат - ' + allNames[ld[0][0]])

По вашему запросу лучший результат - Friends - 3x08 - The One With The Giant Poking Device.ru.txt
